In [1]:
date = 20220930         #current date
prev_date = 20220831    #previous date

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


#=====================================================APPEND=====================================================

location_loan = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\01_MIB_Loans\\"+str(date)+"\\MIB_Loans"+str(date)+".txt"
location_OD = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\05_MIB_OD\\"+str(date)+"\\MIB_OD"+str(date)+".txt"
location_inv = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\06_MIB_Investment\\"+str(date)+"\\MIB_INVESTMENT"+str(date)+".txt"
location_trade = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\07_MIB_Trade\\"+str(date)+"\\MIB_TRADEBILLS"+str(date)+".txt"
location_fcl = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\08_MIB_FCL\\"+str(date)+"\\MIB_FCL"+str(date)+".txt"

loan = pd.read_csv(location_loan, sep = "\t", header = 0, low_memory = False)
OD = pd.read_csv(location_OD, sep = "\t", header = 0)
inv = pd.read_csv(location_inv, sep = "\t", header = 0)
trade = pd.read_csv(location_trade, sep = "\t", header = 0)
fcl = pd.read_csv(location_fcl, sep = "\t", header = 0)

appendP = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\12_Append for YTD\append"+str(prev_date)+".txt", sep = ",", header = 0)


appendP.As_Of_Date = appendP.As_Of_Date.astype(str)
appendP.M_Account_No = appendP.M_Account_No.astype(str)
appendP.Avg_Book_Bal = appendP.Avg_Book_Bal.astype(float)
appendP.M_Interest_Income_Myr = appendP.M_Interest_Income_Myr.astype(float)
appendP.M_Funds_Charge_Myr = appendP.M_Funds_Charge_Myr.astype(float)

appendP['As_Of_Date'].fillna('NA', inplace=True)
appendP['M_Account_No'].fillna('NA', inplace=True)
appendP['Avg_Book_Bal'].fillna(0, inplace=True)
appendP['M_Interest_Income_Myr'].fillna(0, inplace=True)
appendP['M_Funds_Charge_Myr'].fillna(0, inplace=True)


#Loans
loan.columns = loan.columns.str.replace(" ", "_")
loan['M_Account_No'] = loan.M_Account_No.astype(str)
loan['M_Account_No'] = loan.M_Account_No.str[:20]
loan['M_Account_No'].fillna('NA', inplace=True)

#FCL
fcl.columns = fcl.columns.str.replace(" ", "_")
fcl.M_Account_No = fcl.M_Account_No.astype(str)
fcl['M_Account_No'] = fcl.M_Account_No.str[:20]
fcl['M_Account_No'].fillna('NA', inplace=True)

#Investment
inv.columns = inv.columns.str.replace(" ", "_")
inv.M_Account_No = inv.M_Account_No.astype(str)
inv['M_Account_No'] = inv.M_Account_No.str[:20]
inv['M_Account_No'].fillna('NA', inplace=True)

#OD
OD.columns = OD.columns.str.replace(" ", "_")
OD['Account_No'] = OD.Account_No.astype(str)
OD['Account_No'] = OD.Account_No.str[:20]
OD['Common_Coa_Id'] = OD.Common_Coa_Id.astype(int)
OD['Account_No'].fillna('NA', inplace=True)
OD['Common_Coa_Id'].fillna(0, inplace=True)

#Tradebills
trade.columns = trade.columns.str.replace(" ", "_")
trade.Account_No = trade.Account_No.astype(str)
trade['Account_No'] = trade.Account_No.str[:20]
trade['Account_No'].fillna('NA', inplace=True)


loan1 = loan[['As_Of_Date','M_Account_No','Avg_Book_Bal_SUM','M_Interest_Income_Myr_SUM','M_Funds_Charge_Myr_SUM']]
loan1.rename(columns={'Avg_Book_Bal_SUM': 'Avg_Book_Bal','M_Interest_Income_Myr_SUM':'M_Interest_Income_Myr','M_Funds_Charge_Myr_SUM':'M_Funds_Charge_Myr'}, inplace=True)
#loan1.M_Account_No = loan1.M_Account_No.astype(str)

OD_df = OD.iloc[np.where(OD.Common_Coa_Id == 12000)]
OD_df = OD_df[['As_Of_Date','Account_No','Avg_Book_Bal','Interest_Income_Myr','Funds_Charge_Myr']]
OD_df.rename(columns={'Account_No':'M_Account_No','Interest_Income_Myr':'M_Interest_Income_Myr','Funds_Charge_Myr':'M_Funds_Charge_Myr'}, inplace=True)
#OD_df.M_Account_No = OD_df.M_Account_No.astype(str)

FCL = fcl[['As_Of_Date','M_Account_No','Avg_Book_Bal_Myr','M_Interest_Income_Myr']]
FCL.rename(columns={'Avg_Book_Bal_Myr':'Avg_Book_Bal'}, inplace=True)
FCL['M_Funds_Charge_Myr'] = 0
#FCL.M_Account_No = FCL.M_Account_No.astype(str)

INV = inv[['As_Of_Date','M_Account_No','M_Avg_Book_Bal_Myr', 'M_Interest_Income_Myr','M_Funds_Charge_Myr']]
INV.rename(columns={'M_Avg_Book_Bal_Myr':'Avg_Book_Bal'}, inplace = True)
#INV.M_Account_No = INV.M_Account_No.astype(str)

TRADE = trade[['As_Of_Date','Account_No','Avg_Book_Bal_Myr' ,'Interest_Income_Myr','Funds_Charge_Myr']]
TRADE.rename(columns={'Account_No':'M_Account_No','Avg_Book_Bal_Myr':'Avg_Book_Bal','Interest_Income_Myr':'M_Interest_Income_Myr','Funds_Charge_Myr':'M_Funds_Charge_Myr'}, inplace = True)
#TRADE.M_Account_No = TRADE.M_Account_No.astype(str)

frames = [ appendP, loan1, OD_df, INV, TRADE, FCL] 
appendR = pd.concat(frames)


#=====================================================APPEND_AITAB=====================================================

location_HP = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\09_MIB_HP\\"+str(date)+"\MIB_HP"+str(date)+".txt"
location_HPFS = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\09_MIB_HP\\"+str(date)+"\MIB_HPFS"+str(date)+".txt"
location_HPBD = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\09_MIB_HP\\"+str(date)+"\MIB_HPBD"+str(date)+".txt"

HP = pd.read_csv(location_HP, sep = "\t", header = 0)
HPFS = pd.read_csv(location_HPFS, sep = "\t", header = 0)
HPBD = pd.read_csv(location_HPBD, sep = "\t", header = 0)

append_aitabP = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\12_Append for YTD\\append_aitab"+str(prev_date)+".txt", sep = ",", header = 0)

append_aitabP.As_Of_Date = append_aitabP.As_Of_Date.astype(str)
append_aitabP.Account_No = append_aitabP.Account_No.astype(str)
append_aitabP.Avg_Book_Bal = append_aitabP.Avg_Book_Bal.astype(float)
append_aitabP.Interest_Income_Myr = append_aitabP.Interest_Income_Myr.astype(float)
append_aitabP.Fund_Charge_Myr = append_aitabP.Fund_Charge_Myr.astype(float)

append_aitabP['As_Of_Date'].fillna('NA', inplace=True)
append_aitabP['Account_No'].fillna('NA', inplace=True)
append_aitabP['Avg_Book_Bal'].fillna(0, inplace=True)
append_aitabP['Interest_Income_Myr'].fillna(0, inplace=True)
append_aitabP['Fund_Charge_Myr'].fillna(0, inplace=True)

#HP
HP.columns = HP.columns.str.replace(" ", "_")
HP.Account_No = HP.Account_No.astype(str)
HP['Account_No'] = HP.Account_No.str[:20]
HP['Account_No'].fillna('NA', inplace=True)

#HPFS
HPFS.columns = HPFS.columns.str.replace(" ", "_")
HPFS.Account_No = HPFS.Account_No.astype(str)
HPFS['Account_No'] = HPFS.Account_No.str[:20]
HPFS['Account_No'].fillna('NA', inplace=True)

#HPBD
HPBD.columns = HPBD.columns.str.replace(" ", "_")
HPBD.Account_No = HPBD.Account_No.astype(str)
HPBD['Account_No'] = HPBD.Account_No.str[:20]
HPBD['Account_No'].fillna('NA', inplace=True)


MIBHP_1 = HP[['As_Of_Date','Account_No','Avg_Book_Bal','Interest_Income_Myr', 'Fund_Charge_Myr']]
#MIBHP_1.Account_No = MIBHP_1.Account_No.astype(str)

MIBHPFS_1 = HPFS[['As_Of_Date','Account_No','Avg_Book_Bal','Interest_Income_Myr','Fund_Charge_Myr']]
#MIBHPFS_1.Account_No = MIBHPFS_1.Account_No.astype(str)

MIBHPBD_1 = HPBD[['As_Of_Date','Account_No','Avg_Book_Bal','Interest_Income_Myr','Fund_Charge_Myr']]
#MIBHPBD_1.Account_No = MIBHPBD_1.Account_No.astype(str)

a = append_aitabP.append(MIBHP_1, ignore_index = True)
b = a.append(MIBHPFS_1, ignore_index = True)
append_aitab = b.append(MIBHPBD_1, ignore_index = True)

appendaitab = append_aitab


#=====================================================EXPORT=====================================================

appendR.to_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\12_Append for YTD\\append"+str(date)+".txt", index = False)

appendaitab.to_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\12_Append for YTD\\append_aitab"+str(date)+".txt", index = False)


In [10]:
appendR.shape, appendaitab.shape

((6241585, 5), (8074726, 5))

In [11]:
appendR.shape, appendaitab.shape

((5415132, 5), (7046749, 5))